<a href="https://colab.research.google.com/github/LNOI/DeepLearning_IDS_ANN/blob/main/GAN_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install  seaborn
!pip install numpy
!pip install tensorflow
!pip install keras
!pip install wget
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import wget
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [ ]:
cols="""duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,
num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,
count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,
dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate"""
columns=[]
for c in cols.split(','):
    if(c.strip()):
       columns.append(c.strip())
columns.append('target')
# print(len(columns))
attacks_types = {
'normal': 'normal',
'back': 'dos',
'buffer_overflow': 'u2r',
'ftp_write': 'r2l',
'guess_passwd': 'r2l',
'imap': 'r2l',
'ipsweep': 'probe',
'land': 'dos',
'loadmodule': 'u2r',
'multihop': 'r2l',
'neptune': 'dos',
'nmap': 'probe',
'perl': 'u2r',
'phf': 'r2l',
'pod': 'dos',
'portsweep': 'probe',
'rootkit': 'u2r',
'satan': 'probe',
'smurf': 'dos',
'spy': 'r2l',
'teardrop': 'dos',
'warezclient': 'r2l',
'warezmaster': 'r2l',
}
path = "https://github.com/LNOI/DeepLearning_IDS_ANN/raw/main/data/kddcup.data_10_percent.gz"
wget.download(path)
df = pd.read_csv('kddcup.data_10_percent.gz',names=columns)
#Adding Attack Type column
df['Attack Type'] = df.target.apply(lambda r:attacks_types[r[:-1]])
print(df.shape)
num_cols = df._get_numeric_data().columns
cate_cols = list(set(df.columns)-set(num_cols))
cate_cols.remove('target')
cate_cols.remove('Attack Type')
df = df.dropna('columns')# drop columns with NaN
df = df[[col for col in df if df[col].nunique() > 1]]

df.drop('num_root',axis = 1,inplace = True)
df.drop('srv_serror_rate',axis = 1,inplace = True)
df.drop('srv_rerror_rate',axis = 1, inplace=True)
df.drop('dst_host_srv_serror_rate',axis = 1, inplace=True)
df.drop('dst_host_serror_rate',axis = 1, inplace=True)
df.drop('dst_host_rerror_rate',axis = 1, inplace=True)
df.drop('dst_host_srv_rerror_rate',axis = 1, inplace=True)
df.drop('dst_host_same_srv_rate',axis = 1, inplace=True)

#protocol_type feature mapping
pmap = {'icmp':0,'tcp':1,'udp':2}
df['protocol_type'] = df['protocol_type'].map(pmap)

#flag feature mapping
fmap = {'SF':0,'S0':1,'REJ':2,'RSTR':3,'RSTO':4,'SH':5 ,'S1':6 ,'S2':7,'RSTOS0':8,'S3':9 ,'OTH':10}
df['flag'] = df['flag'].map(fmap)

df.drop('service',axis = 1,inplace= True)

from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()

df = df.drop(['target',], axis=1)
print(df.shape)

# Target variable and train set
Y = df[['Attack Type']]
X = df.drop(['Attack Type',], axis=1)

sc = MinMaxScaler()
X = sc.fit_transform(X)

# Split test and train data 

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
Y_train = lb.fit_transform(Y_train)
Y_test = lb.fit_transform(Y_test)
print(X_train.shape, X_test.shape)
print(Y_train.shape, Y_test.shape)
sample = X_train.shape[0]
features = X_train.shape[1]
X_train = np.reshape(X_train,(sample,features,1))
X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))

(494021, 43)


<ipython-input-52-fb9de4317bc6>:46: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only
  df = df.dropna('columns')# drop columns with NaN


(494021, 31)
(330994, 30) (163027, 30)
(330994,) (163027,)


/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
def create_model():
  model = tf.keras.models.Sequential([
      keras.layers.LSTM(30,input_shape=(features,X_train.shape[2]),
                            activation='tanh',recurrent_activation='hard_sigmoid'),
      keras.layers.Dense(1,activation="tanh")
  ])
  model.compile(optimizer='rmsprop',loss='mse', metrics=['accuracy'])
  return model

model = create_model()
model.fit(X_train, 
          Y_train,  
          epochs=1,
          batch_size=64,
          validation_data=(X_test, Y_test))

5172/5172 [==============================] - 163s 31ms/step - loss: 0.0398 - accuracy: 0.9681 - val_loss: 0.0277 - val_accuracy: 0.9803


In [ ]:
start_time = time.time()
Y_train_pred = model.predict(X_train)
end_time = time.time()

10344/10344 [==============================] - 54s 5ms/step


In [ ]:
np.amax(Y_train_pred)

-0.060773276

1

In [ ]:
# Import necessary libraries
import numpy as np
import tensorflow as tf

# Load the real network traffic data
# real_data = X_train

# Create the generator network
generator = tf.keras.Sequential()
generator.add(tf.keras.layers.LSTM(30, input_shape=(X_train.shape[1], X_train.shape[2])))
generator.add(tf.keras.layers.Dense(X_train.shape[1]))

# Create the discriminator network
discriminator = tf.keras.Sequential()
discriminator.add(tf.keras.layers.LSTM(30, input_shape=(X_train.shape[1], X_train.shape[2])))
discriminator.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Compile the discriminator network
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Set the discriminator weights to be non-trainable
for layer in discriminator.layers:
    layer.trainable = False

# Combine the generator and discriminator networks into a GAN
gan = tf.keras.Sequential([generator, discriminator])

# Compile the GAN
gan.compile(loss='binary_crossentropy', optimizer='adam')

# Train the GAN
for epoch in range(100):
    # Generate synthetic data
    synthetic_data = generator.predict(X_train)
    
    # Concatenate the synthetic data with real data
    synthetic_data = np.reshape(synthetic_data,(synthetic_data.shape[0],synthetic_data.shape[1],1))
    print(synthetic_data.shape)
    print(X_train.shape)
    combined_data = np.concatenate((X_train, synthetic_data))
    
    # Create labels for the real and synthetic data
    real_labels = np.ones(X_train.shape[0])
    synthetic_labels = np.zeros(synthetic_data.shape[0])
    
    # Concatenate the labels
    combined_labels = np.concatenate((real_labels, synthetic_labels))
    
    # Train the discriminator on the combined data
    d_loss, d_acc = discriminator.train_on_batch(combined_data, combined_labels)
    
    # Generate noise as input to the generator
    noise = np.random.rand(X_train.shape[0], X_train.shape[1], X_train.shape[2])
    
    # Train the GAN
    g_loss = gan.train_on_batch(noise, real_labels)
    
    # Print loss and accuracy every 10 epochs
    if epoch % 1 == 0:
        print('epoch: {}, discriminator loss: {}, discriminator accuracy: {}, generator loss: {}'.format(epoch+1, d_loss, d_acc, g_loss))

# Save the trained generator network
generator.save('generator.h5')


10344/10344 [==============================] - 48s 5ms/step
(330994, 30, 1)
(330994, 30, 1)
